## Library Import

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Data Load

In [2]:
police_border = gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
police_border.head(3)

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."


In [3]:
security_con = pd.read_csv('./data/9.5개시도_보안등현황.csv')
security_con.head(3)

,year,address,\nsecuritylight_cnt,lon,lat
0,NaN,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467
1,NaN,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181
2,NaN,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711


In [19]:
cctv_con = pd.read_csv('./data/10.5개시도_CCTV설치현황.csv')
cctv_con.head(3)

,address,purpose,cctv_cnt,set_yr,lat,lon
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,NaN,36.605059,127.296286
1,세종특별자치시 조치원읍 남리144,다목적,2,NaN,36.597678,127.302452
2,세종특별자치시 조치원읍 남리 425,다목적,3,NaN,36.594863,127.300492


## ---------------------------------

## ※ 목표 : Q1 ~ Q5 예측 시 X값으로 보안등과 CCTV 개수를 활용
- 각 lon, lat 데이터에 해당하는 관할서를 매칭시킨 결과를 새 컬럼에 표기

### 1. 보안등 현황(security con)

In [32]:
security_con.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 229405 entries, 0 to 229404
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   year       87225 non-null   float64 
 1   address    218426 non-null  object  
 2   light_cnt  229405 non-null  int64   
 3   lon        229405 non-null  float64 
 4   lat        229405 non-null  float64 
 5   geometry   229405 non-null  geometry
 6   jur_stn    229369 non-null  object  
dtypes: float64(3), geometry(1), int64(1), object(2)
memory usage: 12.3+ MB


In [4]:
# light cnt rename
security_con.rename(columns={'\nsecuritylight_cnt' : 'light_cnt'}, inplace=True)

In [5]:
# csv to geopandas
# lon, lat data를 geometry로 변경
security_con['lat'] = security_con['lat'].astype(float)
security_con['lon'] = security_con['lon'].astype(float)
security_con['geometry'] = security_con.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
security_con = gpd.GeoDataFrame(security_con, geometry='geometry')
security_con.crs = {'init':'epsg:4326'}
security_con = security_con.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
security_con.head(3)

,year,address,light_cnt,lon,lat,geometry
0,NaN,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,POINT (127.28769 36.44347)
1,NaN,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,POINT (127.28881 36.44418)
2,NaN,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,POINT (127.28957 36.44471)


In [7]:
# 관할서 리스트 확인
police_border['NAME'].unique()

array(['세종경찰서', '진주경찰서', '창원서부경찰서', '창원중부경찰서', '마산동부경찰서', '마산중부경찰서',
       '진해경찰서', '서울강동경찰서', '서울송파경찰서', '서울수서경찰서', '서울방배경찰서', '서울남대문경찰서',
       '서울서부경찰서', '서울양천경찰서', '서울강서경찰서', '서울성북경찰서', '서울용산경찰서', '서울광진경찰서',
       '서울종암경찰서', '수원남부경찰서', '서울동대문경찰서', '서울성동경찰서', '서울중부경찰서', '서울혜화경찰서',
       '서울중랑경찰서', '서울강남경찰서', '서울서초경찰서', '서울서대문경찰서', '서울종로경찰서', '서울마포경찰서',
       '수원서부경찰서', '수원중부경찰서', '서울관악경찰서', '서울동작경찰서', '서울금천경찰서', '서울구로경찰서',
       '서울영등포경찰서', '서울도봉경찰서', '서울은평경찰서', '서울강북경찰서', '서울노원경찰서'],
      dtype=object)

In [8]:
def detect_police_name(geometry) :
    for i in range(len(police_border['NAME'].unique())) :    # 보안등 위치의 geometry 와 관할서의 geometry를 비교하여
        if geometry.within(police_border.iloc[i].geometry) :             #  매칭된 관할서명을 return하는 함수
            return police_border.iloc[i].NAME            

In [9]:
detect_police_name(security_con.iloc[0].geometry)

'세종경찰서'

In [10]:
from tqdm import tqdm
tqdm.pandas()

In [11]:
# 위에서 만든 함수 적용 및 결과 확인
security_con['jur_stn'] = '' 
security_con['jur_stn'] = security_con.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 229405/229405 [1:29:47<00:00, 42.58it/s]  


In [12]:
security_con.head(3)

,year,address,light_cnt,lon,lat,geometry,jur_stn
0,NaN,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,POINT (127.28769 36.44347),세종경찰서
1,NaN,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,POINT (127.28881 36.44418),세종경찰서
2,NaN,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,POINT (127.28957 36.44471),세종경찰서


In [13]:
# 추후 활용을 위해 csv file로 저장
security_con.to_csv('security_con.csv')

### 2. CCTV 현황(cctv_con)

In [30]:
cctv_con.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 46203 entries, 0 to 46202
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   address   40985 non-null  object  
 1   purpose   46203 non-null  object  
 2   cctv_cnt  46203 non-null  int64   
 3   set_yr    29393 non-null  object  
 4   lat       46203 non-null  float64 
 5   lon       46203 non-null  float64 
 6   geometry  46203 non-null  geometry
 7   jur_stn   46197 non-null  object  
dtypes: float64(2), geometry(1), int64(1), object(4)
memory usage: 2.8+ MB


In [22]:
# csv to geopandas
# lon, lat data를 geometry로 변경
cctv_con['lat'] = cctv_con['lat'].astype(float)
cctv_con['lon'] = cctv_con['lon'].astype(float)
cctv_con['geometry'] = cctv_con.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
cctv_con = gpd.GeoDataFrame(cctv_con, geometry='geometry')
cctv_con.crs = {'init':'epsg:4326'}
cctv_con = cctv_con.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [23]:
cctv_con.head(3)

,address,purpose,cctv_cnt,set_yr,lat,lon,geometry
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,NaN,36.605059,127.296286,POINT (127.29629 36.60506)
1,세종특별자치시 조치원읍 남리144,다목적,2,NaN,36.597678,127.302452,POINT (127.30245 36.59768)
2,세종특별자치시 조치원읍 남리 425,다목적,3,NaN,36.594863,127.300492,POINT (127.30049 36.59486)


In [25]:
## 1과 동일한 작업을 거침
cctv_con['jur_stn'] = ''
cctv_con['jur_stn'] = cctv_con.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 46203/46203 [15:50<00:00, 48.62it/s]


In [26]:
cctv_con.head(3)

,address,purpose,cctv_cnt,set_yr,lat,lon,geometry,jur_stn
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,NaN,36.605059,127.296286,POINT (127.29629 36.60506),세종경찰서
1,세종특별자치시 조치원읍 남리144,다목적,2,NaN,36.597678,127.302452,POINT (127.30245 36.59768),세종경찰서
2,세종특별자치시 조치원읍 남리 425,다목적,3,NaN,36.594863,127.300492,POINT (127.30049 36.59486),세종경찰서


In [29]:
# 추후 활용을 위해 csv file로 저장
cctv_con.to_csv('cctv_con.csv')